In [1]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import warnings
import cv2

warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

PATH = "/kaggle/input/tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")
PATH_TEST = os.path.join(PATH , 'test.csv')

In [2]:
df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)

In [3]:
df_train['image_path'] = PATH + '/'+ df_train['filename'] 
df_test['image_path'] = PATH + '/'+ df_test['filename'] 

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

train_height_list = []
train_length_list = []
for i in range(len(df_train)):
    image = mpimg.imread(df_train["image_path"][i])
    train_height_list.append(image.shape[0])
    train_length_list.append(image.shape[1])

df_train['height'] = train_height_list
df_train['length'] = train_length_list

test_height_list = []
test_length_list = []
for i in range(len(df_test)):
    image = mpimg.imread(df_test["image_path"][i])
    test_height_list.append(image.shape[0])
    test_length_list.append(image.shape[1])

df_test['height'] = test_height_list
df_test['length'] = test_length_list

In [5]:
height_avg = df_train.height.mean()
length_avg = df_train.length.mean()

df_train['size_cat'] = [0]*len(df_train)
for i in range(len(df_train)):
    if df_train['height'][i] <= height_avg:
        if df_train['length'][i] <= length_avg:
            df_train.loc[i, 'size_cat'] = 0
        else:
            df_train.loc[i, 'size_cat'] = 1
    else:
        if df_train['length'][i] <= length_avg:
            df_train.loc[i, 'size_cat'] = 2
        else:
            df_train.loc[i, 'size_cat'] = 3
df_train

,id,filename,distance,image_path,height,length,size_cat
0,0,0.jpg,83110,/kaggle/input/tsp-cv/0.jpg,673,503,2
1,1,1.jpg,1035,/kaggle/input/tsp-cv/1.jpg,222,906,1
2,2,2.jpg,20756,/kaggle/input/tsp-cv/2.jpg,999,810,3
3,3,3.jpg,13286,/kaggle/input/tsp-cv/3.jpg,717,781,3
4,4,4.jpg,13924,/kaggle/input/tsp-cv/4.jpg,884,609,2
...,...,...,...,...,...,...,...
16013,16014,16014.jpg,1803,/kaggle/input/tsp-cv/16014.jpg,748,839,3
16014,16015,16015.jpg,12492,/kaggle/input/tsp-cv/16015.jpg,643,837,3
16015,16016,16016.jpg,1556,/kaggle/input/tsp-cv/16016.jpg,938,550,2
16016,16017,16017.jpg,16507,/kaggle/input/tsp-cv/16017.jpg,744,845,3


In [6]:
df_test['size_cat'] = [0]*len(df_test)
for i in range(len(df_test)):
    if df_test['height'][i] <= height_avg:
        if df_test['length'][i] <= length_avg:
            df_test.loc[i, 'size_cat'] = 0
        else:
            df_test.loc[i, 'size_cat'] = 1
    else:
        if df_test['length'][i] <= length_avg:
            df_test.loc[i, 'size_cat'] = 2
        else:
            df_test.loc[i, 'size_cat'] = 3
df_test

,id,filename,image_path,height,length,size_cat
0,16019,16019.jpg,/kaggle/input/tsp-cv/16019.jpg,395,710,1
1,16020,16020.jpg,/kaggle/input/tsp-cv/16020.jpg,651,425,2
2,16021,16021.jpg,/kaggle/input/tsp-cv/16021.jpg,628,559,2
3,16022,16022.jpg,/kaggle/input/tsp-cv/16022.jpg,367,1005,1
4,16023,16023.jpg,/kaggle/input/tsp-cv/16023.jpg,512,464,0
...,...,...,...,...,...,...
4000,20019,20019.jpg,/kaggle/input/tsp-cv/20019.jpg,499,376,0
4001,20020,20020.jpg,/kaggle/input/tsp-cv/20020.jpg,543,403,0
4002,20021,20021.jpg,/kaggle/input/tsp-cv/20021.jpg,497,678,1
4003,20022,20022.jpg,/kaggle/input/tsp-cv/20022.jpg,605,739,1


In [7]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

In [8]:
def get_model(df_train,df_test):    
    TRAIN_PCT = 0.8
    TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

    df_train_cut = df_train[0:TRAIN_CUT]
    df_validate_cut = df_train[TRAIN_CUT:]
    WIDTH = 299
    HEIGHT = 299

    training_datagen = ImageDataGenerator(
      rescale = 1./255,
      horizontal_flip=True,
      #vertical_flip=True,
      fill_mode='nearest')

    train_generator = training_datagen.flow_from_dataframe(
            dataframe=df_train_cut,
            directory=PATH,
            x_col="filename",
            y_col="distance",
            target_size=(HEIGHT, WIDTH),
            batch_size=32, # Keeping the training batch size small USUALLY increases performance
            class_mode='raw')

    validation_datagen = ImageDataGenerator(rescale = 1./255)

    val_generator = validation_datagen.flow_from_dataframe(
            dataframe=df_validate_cut,
            directory=PATH,
            x_col="filename",
            y_col="distance",
            target_size=(HEIGHT, WIDTH),
            batch_size=256, # Make the validation batch size as large as you have memory for
            class_mode='raw')

    test_datagen = ImageDataGenerator(rescale = 1./255)

    test_generator = test_datagen.flow_from_dataframe(
            dataframe=df_test,
            directory=PATH,
            x_col="filename",
            batch_size=1,
            shuffle=False,
            target_size=(HEIGHT, WIDTH),
            class_mode=None)

    input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

    base_model = Xception(
        include_top=False, weights='imagenet', input_tensor=input_tensor,
        input_shape=None)

    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(2048,activation='relu')(x) 
    x=Dense(2048,activation='relu')(x) 
    model=Model(inputs=base_model.input,outputs=Dense(1)(x))

    # Important, calculate a valid step size for the validation dataset
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
    base_model.trainable = True
    model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError(name="rmse")])
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto',
            restore_best_weights=True)

    history = model.fit(train_generator, epochs=100, steps_per_epoch=STEP_SIZE_TRAIN, 
                        validation_data = val_generator, callbacks=[monitor],
                        verbose = 1, validation_steps=STEP_SIZE_VALID)
    test_generator.reset()
    pred = model.predict(test_generator,steps=len(df_test))
    return pred

In [9]:
pred0 = get_model(df_train[df_train['size_cat'] == 0], df_test[df_test['size_cat'] == 0])
pred1 = get_model(df_train[df_train['size_cat'] == 1], df_test[df_test['size_cat'] == 1])
pred2 = get_model(df_train[df_train['size_cat'] == 2], df_test[df_test['size_cat'] == 2])
pred3 = get_model(df_train[df_train['size_cat'] == 3], df_test[df_test['size_cat'] == 3])

Found 3170 validated image filenames.
Found 793 validated image filenames.
Found 1001 validated image filenames.


2022-04-18 20:42:06.462948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 20:42:06.570088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 20:42:06.571101: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 20:42:06.572752: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

83697664/83683744 [==============================] - 0s 0us/step


2022-04-18 20:42:10.558846: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-04-18 20:42:15.737923: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


99/99 [==============================] - ETA: 0s - loss: 118291480.0000 - rmse: 10876.1885

2022-04-18 20:43:36.574166: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.65GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-18 20:43:36.574737: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


99/99 [==============================] - 91s 807ms/step - loss: 118291480.0000 - rmse: 10876.1885 - val_loss: 2268794624.0000 - val_rmse: 47631.8672
Epoch 2/100
99/99 [==============================] - 77s 773ms/step - loss: 22329128.0000 - rmse: 4725.3706 - val_loss: 2369933568.0000 - val_rmse: 48681.9648
Epoch 3/100
99/99 [==============================] - 77s 772ms/step - loss: 20672612.0000 - rmse: 4546.7144 - val_loss: 14751043.0000 - val_rmse: 3840.7087
Epoch 4/100
99/99 [==============================] - 77s 775ms/step - loss: 15212504.0000 - rmse: 3900.3210 - val_loss: 4604370.0000 - val_rmse: 2145.7795
Epoch 5/100
99/99 [==============================] - 77s 775ms/step - loss: 14619155.0000 - rmse: 3823.5002 - val_loss: 127077736.0000 - val_rmse: 11272.8760
Epoch 6/100
99/99 [==============================] - 78s 782ms/step - loss: 8041803.0000 - rmse: 2835.8074 - val_loss: 4284938.5000 - val_rmse: 2070.0093
Epoch 7/100
99/99 [==============================] - 77s 775ms/step -

In [10]:
# df_submit = pd.DataFrame()
# id_list = df_test[df_test['size_cat'] == 0].id.tolist() + df_test[df_test['size_cat'] == 1].id.tolist()
# pred_list = list(pred0.flatten()) + list(pred1.flatten())
# df_submit['id'] = id_list
# df_submit['distance'] = pred_list
# df_submit

In [11]:
df_submit = pd.DataFrame()
id_list = df_test[df_test['size_cat'] == 0].id.tolist() + df_test[df_test['size_cat'] == 1].id.tolist() + df_test[df_test['size_cat'] == 2].id.tolist() + df_test[df_test['size_cat'] == 3].id.tolist()
pred_list = list(pred0.flatten()) + list(pred1.flatten()) + list(pred2.flatten()) + list(pred3.flatten())
df_submit['id'] = id_list
df_submit['distance'] = pred_list

In [12]:
df_submit.to_csv("submit.csv",index=False)

In [13]:
# id_list = df_test[df_test['size_cat'] == 0].id.tolist() + df_test[df_test['size_cat'] == 1].id.tolist() + df_test[df_test['size_cat'] == 2].id.tolist() + df_test[df_test['size_cat'] == 3].id.tolist()
# len(id_list)

In [14]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np
# plt.rcParams['figure.figsize'] = (15,9)
# def select_blue(image):
#     # Grab the x and y size and make a copy of the image
#     ysize = image.shape[0]
#     xsize = image.shape[1]
#     color_select = np.copy(image)

#     # Define color selection criteria
#     ###### MODIFY THESE VARIABLES TO MAKE YOUR COLOR SELECTION
#     red_threshold = -1
#     green_threshold = -1
#     blue_threshold = 135
#     ######

#     rgb_threshold = [red_threshold, green_threshold, blue_threshold]

#     # Do a boolean or with the "|" character to identify
#     # pixels below the thresholds
#     thresholds = (image[:,:,0] < rgb_threshold[0]) \
#                 | (image[:,:,1] < rgb_threshold[1]) \
#                 | (image[:,:,2] < rgb_threshold[2])
#     color_select[thresholds] = [0,0,0]
#     image = color_select
    
#     return image

We want to use early stopping.  To do this, we need a validation set.  We will break the data into 80 percent test data and 20 validation.  

In [15]:
# TRAIN_PCT = 0.9
# TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

# df_train_cut = df_train[0:TRAIN_CUT]
# df_validate_cut = df_train[TRAIN_CUT:]

# print(f"Training size: {len(df_train_cut)}")
# print(f"Validate size: {len(df_validate_cut)}")

Next, we create the generators that will provide the images to the neural network as it is trained.  We normalize the images so that the RGB colors between 0-255 become ratios between 0 and 1.  We also use the **flow_from_dataframe** generator to connect the Pandas dataframe to the actual image files. We see here a straightforward implementation; you might also wish to use some of the image transformations provided by the data generator.

The **HEIGHT** and **WIDTH** constants specify the dimensions that the image will be scaled (or expanded) to. It is probably not a good idea to expand the images.

In [16]:
# def line_detect_possible(img):
#     L = len(img)
#     H = len(img[0])
#     gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
#     gray = gray.astype(np.uint8)
#     edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    
#     lines = cv2.HoughLinesP(edges, 1, np.pi/720, 30, minLineLength=20, maxLineGap=8)
#     img = np.zeros( ( L, H, 3), np.uint8 )
#     for line in lines:
#         x1, y1, x2, y2 = line[0]
#         cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
#     return img.astype('float64')

In [17]:
# import tensorflow as tf
# import keras_preprocessing
# from keras_preprocessing import image
# from keras_preprocessing.image import ImageDataGenerator

# HEIGHT = 299
# WIDTH = 299

# training_datagen = ImageDataGenerator(
#   rescale = 1./255,
#   horizontal_flip=True,
#   #vertical_flip=True,
#   fill_mode='nearest'
# #   preprocessing_function = select_blue
# )


# train_generator = training_datagen.flow_from_dataframe(
#         dataframe=df_train_cut,
#         directory=PATH,
#         x_col="filename",
#         y_col="distance",
#         target_size=(HEIGHT, WIDTH),
#         batch_size=32, # Keeping the training batch size small USUALLY increases performance
#         class_mode='raw')

# validation_datagen = ImageDataGenerator(
#     rescale = 1./255
# #     preprocessing_function = select_blu
# )

# val_generator = validation_datagen.flow_from_dataframe(
#         dataframe=df_validate_cut,
#         directory=PATH,
#         x_col="filename",
#         y_col="distance",
#         target_size=(HEIGHT, WIDTH),
#         batch_size=256, # Make the validation batch size as large as you have memory for
#         class_mode='raw')


# test_datagen = ImageDataGenerator(
#     rescale = 1./255
# #     preprocessing_function = select_blue
# )

# test_generator = validation_datagen.flow_from_dataframe(
#         dataframe=df_test,
#         directory=PATH,
#         x_col="filename",
#         batch_size=1,
#         shuffle=False,
#         target_size=(HEIGHT, WIDTH),
#         class_mode=None)

## Transfer Learning

We will now use a Xception neural network as a basis for our neural network.  We will redefine both the input shape and output of the ResNet model, so we will not transfer the weights.  Since we redefine the input; the weights are of minimal value.  We begin by loading, from Keras, the Xception network.

In [18]:
# from tensorflow.keras.applications import Xception
# from tensorflow.keras.layers import Input
# from tensorflow.keras.models import Sequential

# input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

# base_model = Xception(
#     include_top=False, weights=None, input_tensor=input_tensor,
#     input_shape=None)

Now we must add a few layers to the end of the neural network so that it becomes a regression model.

In [19]:
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model

# x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(2048,activation='relu')(x) 
# x=Dense(2048,activation='relu')(x) 
# model=Model(inputs=base_model.input,outputs=Dense(1)(x))

Now we train just like before, the only difference is that we do not define the entire neural network here.

In [20]:
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.metrics import RootMeanSquaredError

# # Important, calculate a valid step size for the validation dataset
# STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

# model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError(name="rmse")])
# monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto',
#         restore_best_weights=True)

# history = model.fit(train_generator, epochs=100, steps_per_epoch=50, 
#                     validation_data = val_generator, callbacks=[monitor],
#                     verbose = 1, validation_steps=STEP_SIZE_VALID)

In [21]:
# test_generator.reset()
# pred = model.predict(test_generator,steps=len(df_test))

# df_submit = pd.DataFrame({'id':df_test['id'],'distance':pred.flatten()})
# df_submit.to_csv("submit.csv",index=False)